<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/QUANT018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安装必要的依赖项
!pip install yfinance mplfinance  # 使用pip安装yfinance和mplfinance库

import sys  # 导入sys库，用于访问系统特定的参数和函数
import os   # 导入os库，用于与操作系统进行交互

import yfinance as yf  # 导入yfinance库，用于下载股票数据
import pandas as pd  # 导入pandas库，用于数据处理

def download_data(ticker, start_date, end_date):
    """
    下载股票数据并处理多重索引

    参数：
    ticker (str): 股票代码
    start_date (str): 开始日期
    end_date (str): 结束日期

    返回：
    pandas.DataFrame: 股票数据
    """
    df = yf.download(ticker, start=start_date, end=end_date)  # 使用yfinance下载股票数据
    df.columns = df.columns.droplevel(1)  # 删除第二级索引(ticker名称)
    return df  # 返回处理后的数据

# 设置参数
ticker = 'AAPL'  # 设置股票代码为AAPL（苹果公司）
start_date = '2020-01-01'  # 设置开始日期为2020年1月1日
end_date = '2023-10-26'  # 设置结束日期为2023年10月26日

# 下载数据
df = download_data(ticker, start_date, end_date)  # 调用函数下载股票数据
print(df.columns)  # 打印数据帧的列名
print(df.head(100))  # 打印数据帧的前100行
df.to_csv('stock_1d.csv')  # 将数据帧保存为名为'stock_1d.csv'的CSV文件

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
Price           Close       High        Low       Open     Volume
Date                                                             
2020-01-02  72.716064  72.776591  71.466805  71.721011  135480400
2020-01-03  72.009125  72.771752  71.783969  71.941336  146322800
2020-01-06  72.582901  72.621639  70.876068  71.127858  118387200
2020-01-07  72.241547  72.849224  72.021231  72.592594  108872000
2020-01-08  73.403641  73.706271  71.943751  71.943751  132079200
...               ...        ...        ...        ...        ...
2020-05-19  76.198235  77.507376  76.166600  76.658136  101729600
2020-05-20  77.680153  77.750715  77.020707  77.059642  111504800
2020-05-21  77.100990  78.084070  76.862518  77.541428  102688800
2020-05-22  77.597420  77.680153  76.736008  76.838205   81803200
2020-05-26  77.071800  78.899248  77.015830  78.719182  125522000

[100 rows x 5 columns]


In [2]:
import pandas as pd
import numpy as np
from typing import List, Dict, Union, Tuple
from scipy import stats

class AlphaFeatures:
    """
    股票技术指标特征工程类
    """
    def __init__(self, df: pd.DataFrame):
        """
        初始化

        参数:
        df: pd.DataFrame - 包含OHLCV数据的DataFrame
        """
        self.df = df.copy()
        # 确保列名标准化
        self.df.columns = [col.lower() for col in self.df.columns]
        # 添加vwap列
        self.df['vwap'] = (self.df['high'] + self.df['low'] + self.df['close']) / 3

    def get_alpha360_features(self, windows: int = 60) -> pd.DataFrame:
        """
        计算Alpha360特征
        """
        features = {}
        price_cols = ['close', 'open', 'high', 'low', 'vwap']

        for col in price_cols:
            # 当前值
            features[f'{col.upper()}0'] = self.df[col] / self.df['close']

            # 历史值
            for i in range(1, windows):
                features[f'{col.upper()}{i}'] = self.df[col].shift(i) / self.df['close']

        # 成交量特征
        features['VOLUME0'] = self.df['volume'] / (self.df['volume'] + 1e-12)
        for i in range(1, windows):
            features[f'VOLUME{i}'] = self.df['volume'].shift(i) / (self.df['volume'] + 1e-12)

        return pd.DataFrame(features, index=self.df.index)

    def get_kbar_features(self) -> pd.DataFrame:
        """
        计算K线特征
        """
        features = {}
        df = self.df

        # 基础K线特征
        features['KMID'] = (df['close'] - df['open']) / df['open']
        features['KLEN'] = (df['high'] - df['low']) / df['open']
        features['KMID2'] = (df['close'] - df['open']) / (df['high'] - df['low'] + 1e-12)

        # 上影线和下影线特征
        features['KUP'] = (df['high'] - df[['open', 'close']].max(axis=1)) / df['open']
        features['KUP2'] = (df['high'] - df[['open', 'close']].max(axis=1)) / (df['high'] - df['low'] + 1e-12)
        features['KLOW'] = (df[['open', 'close']].min(axis=1) - df['low']) / df['open']
        features['KLOW2'] = (df[['open', 'close']].min(axis=1) - df['low']) / (df['high'] - df['low'] + 1e-12)

        # 位移特征
        features['KSFT'] = (2 * df['close'] - df['high'] - df['low']) / df['open']
        features['KSFT2'] = (2 * df['close'] - df['high'] - df['low']) / (df['high'] - df['low'] + 1e-12)

        return pd.DataFrame(features, index=self.df.index)

    def get_price_volume_features(self, windows: List[int] = [0,1,2,3,4]) -> pd.DataFrame:
        """
        计算价格和成交量特征
        """
        features = {}
        fields = ['open', 'high', 'low', 'close', 'vwap']

        for field in fields:
            for d in windows:
                if d == 0:
                    features[f'{field.upper()}{d}'] = self.df[field] / self.df['close']
                else:
                    features[f'{field.upper()}{d}'] = self.df[field].shift(d) / self.df['close']

        for d in windows:
            if d == 0:
                features[f'VOLUME{d}'] = self.df['volume'] / (self.df['volume'] + 1e-12)
            else:
                features[f'VOLUME{d}'] = self.df['volume'].shift(d) / (self.df['volume'] + 1e-12)

        return pd.DataFrame(features, index=self.df.index)

    def get_rolling_features(self, windows: List[int] = [5, 10, 20, 30, 60]) -> pd.DataFrame:
        """
        计算滚动窗口特征
        """
        features = {}
        df = self.df

        for d in windows:
            # ROC - 价格变化率
            features[f'ROC{d}'] = df['close'].shift(d) / df['close']

            # MA - 移动平均
            features[f'MA{d}'] = df['close'].rolling(d).mean() / df['close']

            # STD - 标准差
            features[f'STD{d}'] = df['close'].rolling(d).std() / df['close']

            # BETA - 斜率
            features[f'BETA{d}'] = self._calculate_slope(df['close'], d) / df['close']

            # RSQR - R方值
            features[f'RSQR{d}'] = self._calculate_rsquare(df['close'], d)

            # RESI - 残差
            features[f'RESI{d}'] = self._calculate_residuals(df['close'], d) / df['close']

            # MAX/MIN/Quantile
            features[f'MAX{d}'] = df['high'].rolling(d).max() / df['close']
            features[f'MIN{d}'] = df['low'].rolling(d).min() / df['close']
            features[f'QTLU{d}'] = df['close'].rolling(d).quantile(0.8) / df['close']
            features[f'QTLD{d}'] = df['close'].rolling(d).quantile(0.2) / df['close']

            # RANK - 排名
            features[f'RANK{d}'] = df['close'].rolling(d).apply(
                lambda x: stats.percentileofscore(x.values, x.iloc[-1])
            ) / 100

            # RSV - 价格位置
            features[f'RSV{d}'] = (df['close'] - df['low'].rolling(d).min()) / \
                                (df['high'].rolling(d).max() - df['low'].rolling(d).min() + 1e-12)

            # IMAX/IMIN/IMXD - Aroon指标
            features[f'IMAX{d}'] = df['high'].rolling(d).apply(
                lambda x: len(x) - 1 - np.argmax(x.values)
            ) / d
            features[f'IMIN{d}'] = df['low'].rolling(d).apply(
                lambda x: len(x) - 1 - np.argmin(x.values)
            ) / d
            features[f'IMXD{d}'] = features[f'IMAX{d}'] - features[f'IMIN{d}']

            # 相关性特征
            features[f'CORR{d}'] = df['close'].rolling(d).corr(np.log(df['volume'] + 1))
            features[f'CORD{d}'] = (df['close'] / df['close'].shift(1)).rolling(d).corr(
                np.log(df['volume'] / df['volume'].shift(1) + 1)
            )

            # 计数特征
            price_up = (df['close'] > df['close'].shift(1)).astype(int)
            features[f'CNTP{d}'] = price_up.rolling(d).mean()
            features[f'CNTN{d}'] = (1 - price_up).rolling(d).mean()
            features[f'CNTD{d}'] = features[f'CNTP{d}'] - features[f'CNTN{d}']

            # RSI类特征
            price_diff = df['close'] - df['close'].shift(1)
            gain = np.where(price_diff > 0, price_diff, 0)
            loss = np.where(price_diff < 0, -price_diff, 0)
            abs_price_diff = abs(price_diff)

            features[f'SUMP{d}'] = pd.Series(gain).rolling(d).sum() / \
                                 (pd.Series(abs_price_diff).rolling(d).sum() + 1e-12)
            features[f'SUMN{d}'] = pd.Series(loss).rolling(d).sum() / \
                                 (pd.Series(abs_price_diff).rolling(d).sum() + 1e-12)
            features[f'SUMD{d}'] = features[f'SUMP{d}'] - features[f'SUMN{d}']

            # 成交量特征
            features[f'VMA{d}'] = df['volume'].rolling(d).mean() / (df['volume'] + 1e-12)
            features[f'VSTD{d}'] = df['volume'].rolling(d).std() / (df['volume'] + 1e-12)

            # 成交量加权波动率
            price_change = abs(df['close'] / df['close'].shift(1) - 1) * df['volume']
            features[f'WVMA{d}'] = price_change.rolling(d).std() / \
                                 (price_change.rolling(d).mean() + 1e-12)

            # 成交量RSI类特征
            volume_diff = df['volume'] - df['volume'].shift(1)
            volume_gain = np.where(volume_diff > 0, volume_diff, 0)
            volume_loss = np.where(volume_diff < 0, -volume_diff, 0)
            abs_volume_diff = abs(volume_diff)

            features[f'VSUMP{d}'] = pd.Series(volume_gain).rolling(d).sum() / \
                                  (pd.Series(abs_volume_diff).rolling(d).sum() + 1e-12)
            features[f'VSUMN{d}'] = pd.Series(volume_loss).rolling(d).sum() / \
                                  (pd.Series(abs_volume_diff).rolling(d).sum() + 1e-12)
            features[f'VSUMD{d}'] = features[f'VSUMP{d}'] - features[f'VSUMN{d}']

        return pd.DataFrame(features, index=self.df.index)

    def _calculate_slope(self, series: pd.Series, window: int) -> pd.Series:
        """计算滚动窗口的斜率"""
        def _slope(x):
            x = x.values  # 转换为numpy数组
            y = np.arange(len(x))
            A = np.vstack([y, np.ones(len(x))]).T
            return np.linalg.lstsq(A, x, rcond=None)[0][0]
        return series.rolling(window).apply(_slope)

    def _calculate_rsquare(self, series: pd.Series, window: int) -> pd.Series:
        """计算滚动窗口的R方值"""
        def _rsquare(x):
            x = x.values  # 转换为numpy数组
            y = np.arange(len(x))
            A = np.vstack([y, np.ones(len(x))]).T
            b = np.linalg.lstsq(A, x, rcond=None)[0]
            y_pred = A @ b
            ss_tot = np.sum((x - x.mean()) ** 2)
            ss_res = np.sum((x - y_pred) ** 2)
            return 1 - ss_res / ss_tot
        return series.rolling(window).apply(_rsquare)

    def _calculate_residuals(self, series: pd.Series, window: int) -> pd.Series:
        """计算滚动窗口的残差"""
        def _residuals(x):
            x = x.values  # 转换为numpy数组
            y = np.arange(len(x))
            A = np.vstack([y, np.ones(len(x))]).T
            b = np.linalg.lstsq(A, x, rcond=None)[0]
            y_pred = A @ b
            return x[-1] - y_pred[-1]
        return series.rolling(window).apply(_residuals)

    def generate_features(self,
                         alpha360: bool = True,
                         kbar: bool = True,
                         price_volume: bool = True,
                         rolling: bool = True,
                         alpha360_windows: int = 60,
                         price_volume_windows: List[int] = [0,1,2,3,4],
                         rolling_windows: List[int] = [5,10,20,30,60]) -> pd.DataFrame:
        """
        生成所有特征

        参数:
        alpha360: bool - 是否生成Alpha360特征
        kbar: bool - 是否生成K线特征
        price_volume: bool - 是否生成价格成交量特征
        rolling: bool - 是否生成滚动窗口特征
        alpha360_windows: int - Alpha360特征的窗口大小
        price_volume_windows: List[int] - 价格成交量特征的窗口大小列表
        rolling_windows: List[int] - 滚动窗口特征的窗口大小列表

        返回:
        pd.DataFrame - 特征数据
        """
        # 检查数据长度是否足够
        min_required_length = max([alpha360_windows] + rolling_windows) if rolling else alpha360_windows
        if len(self.df) < min_required_length:
            raise ValueError(f"Data length ({len(self.df)}) is less than minimum required length ({min_required_length})")

        feature_dfs = []

        if alpha360:
            feature_dfs.append(self.get_alpha360_features(alpha360_windows))
        if kbar:
            feature_dfs.append(self.get_kbar_features())
        if price_volume:
            feature_dfs.append(self.get_price_volume_features(price_volume_windows))
        if rolling:
            feature_dfs.append(self.get_rolling_features(rolling_windows))

        # 一次性合并所有特征
        if feature_dfs:
            result = pd.concat(feature_dfs, axis=1)
            # 删除全为NA的列
            result = result.dropna(axis=1, how='all')
            # 删除包含NA的行
            result = result.dropna(axis=0, how='any')
            return result

        return pd.DataFrame()

# 使用示例
if __name__ == "__main__":
    # 读取数据
    df = pd.read_csv('stock_1d.csv', index_col='Date', parse_dates=True)
    print("Data shape:", df.shape)

    # 初始化特征工程类
    af = AlphaFeatures(df)

    # 生成所有特征
    features_df = af.generate_features()

    # 保存带特征的数据
    features_df.to_csv('stock_1d_features.csv')

    print("Feature shape:", features_df.shape)
    print("\nFeature columns:", features_df.columns.tolist())

Data shape: (961, 5)
Feature shape: (901, 514)

Feature columns: ['CLOSE0', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'CLOSE5', 'CLOSE6', 'CLOSE7', 'CLOSE8', 'CLOSE9', 'CLOSE10', 'CLOSE11', 'CLOSE12', 'CLOSE13', 'CLOSE14', 'CLOSE15', 'CLOSE16', 'CLOSE17', 'CLOSE18', 'CLOSE19', 'CLOSE20', 'CLOSE21', 'CLOSE22', 'CLOSE23', 'CLOSE24', 'CLOSE25', 'CLOSE26', 'CLOSE27', 'CLOSE28', 'CLOSE29', 'CLOSE30', 'CLOSE31', 'CLOSE32', 'CLOSE33', 'CLOSE34', 'CLOSE35', 'CLOSE36', 'CLOSE37', 'CLOSE38', 'CLOSE39', 'CLOSE40', 'CLOSE41', 'CLOSE42', 'CLOSE43', 'CLOSE44', 'CLOSE45', 'CLOSE46', 'CLOSE47', 'CLOSE48', 'CLOSE49', 'CLOSE50', 'CLOSE51', 'CLOSE52', 'CLOSE53', 'CLOSE54', 'CLOSE55', 'CLOSE56', 'CLOSE57', 'CLOSE58', 'CLOSE59', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'OPEN5', 'OPEN6', 'OPEN7', 'OPEN8', 'OPEN9', 'OPEN10', 'OPEN11', 'OPEN12', 'OPEN13', 'OPEN14', 'OPEN15', 'OPEN16', 'OPEN17', 'OPEN18', 'OPEN19', 'OPEN20', 'OPEN21', 'OPEN22', 'OPEN23', 'OPEN24', 'OPEN25', 'OPEN26', 'OPEN27', 'OPEN28', 'OPE

In [3]:
features_df.head(1)

,CLOSE0,CLOSE1,CLOSE2,CLOSE3,CLOSE4,CLOSE5,CLOSE6,CLOSE7,CLOSE8,CLOSE9,...,IMIN60,IMXD60,CORR60,CORD60,CNTP60,CNTN60,CNTD60,VMA60,VSTD60,WVMA60
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-30,1.0,0.972254,1.014246,0.963542,0.968879,0.880538,0.899651,0.960637,0.968055,0.992347,...,0.083333,0.45,-0.805677,-0.391011,0.45,0.55,-0.1,1.180844,0.560479,1.428129


In [4]:
import numpy as np  # 导入numpy库，用于数值计算
import pandas as pd  # 导入pandas库，用于数据处理
import plotly.graph_objects as go  # 导入plotly.graph_objects库，用于创建图形对象
from plotly.subplots import make_subplots  # 导入plotly.subplots库，用于创建子图
from dataclasses import dataclass  # 导入dataclasses库，用于创建数据类
from typing import List, Tuple, Optional, Dict  # 导入typing库，用于类型注解


@dataclass  # 使用dataclass装饰器创建数据类
class FractalPoint:
    """分型点数据类"""
    type: int  # 1: 顶分型, -1: 底分型, 0: 起始/结束点
    time: str  # 分型点时间
    price: float  # 分型点价格
    offset: int  # 分型点偏移量

@dataclass  # 使用dataclass装饰器创建数据类
class PivotRange:
    """中枢范围数据类"""
    x_range: Tuple[float, float]  # 时间范围
    price_range: Tuple[float, float]  # 价格范围


class TechnicalAnalysis:
    """技术分析类"""

    def __init__(self, data: pd.DataFrame):
        """初始化技术分析类"""
        self.validate_data(data)  # 验证输入数据
        self.data = data.copy()  # 复制输入数据，避免修改原始数据
        # 保持原始列名不变
        self.column_map = {  # 定义列名映射字典
            'high': 'High',
            'low': 'Low',
            'open': 'Open',
            'close': 'Close',
            'volume': 'Volume'
        }

    @staticmethod
    def validate_data(data: pd.DataFrame) -> None:
        """验证输入数据的完整性"""
        required_columns = ['High', 'Low', 'Open', 'Close', 'Volume']  # 定义必需的列名
        if not all(col in data.columns for col in required_columns):  # 检查所有必需的列是否存在
            raise ValueError(f"Data must contain columns: {required_columns}")  # 如果缺少列，则抛出异常
        if len(data) < 3:  # 检查数据长度是否足够
            raise ValueError("Need at least 3 k-lines for analysis")  # 如果数据长度不足，则抛出异常


    def adjust_by_containment(self, k_data: pd.DataFrame = None) -> pd.DataFrame:
        """处理K线包含关系"""
        if k_data is None:  # 如果没有提供K线数据，则使用原始数据
            k_data = self.data.copy()  # 复制原始数据

        result = []  # 初始化结果列表
        curr_k = k_data.iloc[0].copy()  # 初始化当前K线
        trend = 0  # 初始化趋势

        for i in range(1, len(k_data)):  # 遍历K线数据
            next_k = k_data.iloc[i]  # 获取下一根K线
            contained = (  # 判断是否存在包含关系
                (curr_k['High'] >= next_k['High'] and curr_k['Low'] <= next_k['Low']) or
                (curr_k['High'] <= next_k['High'] and curr_k['Low'] >= next_k['Low'])
            )

            if contained:  # 如果存在包含关系
                if trend >= 0:  # 如果趋势向上或初始状态
                    curr_k['High'] = max(curr_k['High'], next_k['High'])  # 更新当前K线最高价
                    curr_k['Low'] = max(curr_k['Low'], next_k['Low'])  # 更新当前K线最低价
                else:  # 如果趋势向下
                    curr_k['High'] = min(curr_k['High'], next_k['High'])  # 更新当前K线最高价
                    curr_k['Low'] = min(curr_k['Low'], next_k['Low'])  # 更新当前K线最低价
            else:  # 如果不存在包含关系
                result.append({  # 添加当前K线到结果列表
                    'Open': curr_k['Open'],
                    'High': curr_k['High'],
                    'Low': curr_k['Low'],
                    'Close': curr_k['Close'],
                    'Volume': curr_k['Volume'],
                    'Date': k_data.index[i-1]
                })

                if next_k['High'] > curr_k['High'] and next_k['Low'] > curr_k['Low']:  # 如果下一根K线高于当前K线
                    trend = 1  # 设置趋势为向上
                elif next_k['High'] < curr_k['High'] and next_k['Low'] < curr_k['Low']:  # 如果下一根K线低于当前K线
                    trend = -1  # 设置趋势为向下

                curr_k = next_k.copy()  # 更新当前K线

        result.append({  # 添加最后一根K线到结果列表
            'Open': curr_k['Open'],
            'High': curr_k['High'],
            'Low': curr_k['Low'],
            'Close': curr_k['Close'],
            'Volume': curr_k['Volume'],
            'Date': k_data.index[-1]
        })

        df = pd.DataFrame(result)  # 将结果列表转换为DataFrame
        df.set_index('Date', inplace=True)  # 设置日期为索引
        return df  # 返回处理后的K线数据

    def get_fx(self, k_data: pd.DataFrame = None) -> Tuple[List[int], List[str], pd.DataFrame, List[float], List[int]]:
        """识别分型"""
        if k_data is None:  # 如果没有提供K线数据，则使用处理后的K线数据
            k_data = self.adjust_by_containment()  # 使用处理后的K线数据

        fractals = []  # 初始化分型列表

        # 添加起始点
        fractals.append(FractalPoint(  # 添加起始分型点
            type=0,
            time=k_data.index[0].strftime("%Y-%m-%d %H:%M:%S"),
            price=(k_data['Low'].iloc[0] + k_data['High'].iloc[0]) / 2,
            offset=0
        ))

        # 识别分型
        for i in range(1, len(k_data)-1):  # 遍历K线数据
            # 正确获取三根K线
            k1 = k_data.iloc[i-1]
            k2 = k_data.iloc[i]
            k3 = k_data.iloc[i+1]

            # 顶分型
            if (k1['High'] <= k2['High'] and k2['High'] > k3['High']):  # 如果当前K线最高价高于前后K线最高价
                fractals.append(FractalPoint(  # 添加顶分型点
                    type=1,
                    time=k_data.index[i].strftime("%Y-%m-%d %H:%M:%S"),
                    price=k2['High'],
                    offset=i
                ))

            # 底分型
            elif (k1['Low'] >= k2['Low'] and k2['Low'] < k3['Low']):  # 如果当前K线最低价低于前后K线最低价
                fractals.append(FractalPoint(  # 添加底分型点
                    type=-1,
                    time=k_data.index[i].strftime("%Y-%m-%d %H:%M:%S"),
                    price=k2['Low'],
                    offset=i
                ))

        # 添加结束点
        fractals.append(FractalPoint(  # 添加结束分型点
            type=0,
            time=k_data.index[-1].strftime("%Y-%m-%d %H:%M:%S"),
            price=(k_data['Low'].iloc[-1] + k_data['High'].iloc[-1]) / 2,
            offset=len(k_data)-1
        ))

        # 构建返回数据
        fx_type = [f.type for f in fractals]  # 获取分型类型列表
        fx_time = [f.time for f in fractals]  # 获取分型时间列表
        fx_data = k_data.loc[[k_data.index[f.offset] for f in fractals]]  # 获取分型数据DataFrame
        fx_plot = [f.price for f in fractals]  # 获取分型价格列表
        fx_offset = [f.offset for f in fractals]  # 获取分型偏移列表

        return fx_type, fx_time, fx_data, fx_plot, fx_offset  # 返回分型类型、时间、数据、价格、偏移列表




    def create_label_series(self, k_data: pd.DataFrame = None) -> pd.Series:
        """
        创建标签序列

        参数:
        k_data: pd.DataFrame - K线数据

        返回:
        pd.Series - 标签序列
        """
        if k_data is None:
            k_data = self.adjust_by_containment()

        # 初始化标签序列为 'ING'
        labels = pd.Series('ING', index=k_data.index)

        # 识别分型
        fx_type, fx_time, fx_data, fx_plot, fx_offset = self.get_fx(k_data)

        # 将分型信息转换为字典，键为日期，值为分型类型
        fx_dict = {pd.to_datetime(time): type_ for time, type_ in zip(fx_time, fx_type)}

        # 标记分型
        for date, type_ in fx_dict.items():
            if type_ == 1:  # 顶分型
                labels[date] = 'TOP'
            elif type_ == -1:  # 底分型
                labels[date] = 'BOTTOM'

        return labels

def main():
    try:
        # 读取数据
        data = pd.read_csv('stock_1d.csv',
                          index_col='Date',
                          parse_dates=True)

        print("数据加载成功，样本数:", len(data))

        # 创建技术分析对象
        ta = TechnicalAnalysis(data)

        # 处理包含关系并获取标签
        adjusted_k_data = ta.adjust_by_containment(data)
        labels = ta.create_label_series(adjusted_k_data)

        # 将标签合并到原始数据中
        data['label'] = labels

        # 保存带标签的数据
        data.to_csv('stock_1d_label.csv')

        print("\n标签统计:")
        print(data['label'].value_counts())

        # 打印部分带标签的数据
        print("\n带标签的数据示例:")
        print(data[data['label'].isin(['TOP', 'BOTTOM'])].head())

    except FileNotFoundError:
        print("错误: 未找到文件 'stock_1d.csv'，请确保文件存在于正确的路径")
    except Exception as e:
        print(f"处理过程中出现错误: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

数据加载成功，样本数: 961

标签统计:
label
ING       451
BOTTOM    173
TOP       173
Name: count, dtype: int64

带标签的数据示例:
                Close       High        Low       Open     Volume   label
Date                                                                     
2020-01-06  72.582901  72.621639  70.876068  71.127858  118387200  BOTTOM
2020-01-14  75.701210  76.885104  75.577742  76.674475  161954400     TOP
2020-01-15  75.376808  76.383964  74.943439  75.500284  121923600  BOTTOM
2020-01-22  76.916573  77.470986  76.822148  77.129618  101832400     TOP
2020-01-23  77.287003  77.366894  76.420265  76.969846  104472000  BOTTOM


In [5]:
import pandas as pd

# 读取两个 CSV 文件
features_df = pd.read_csv('stock_1d_features.csv', index_col='Date', parse_dates=True)
label_df = pd.read_csv('stock_1d_label.csv', index_col='Date', parse_dates=True)

# 合并两个 DataFrame
features_label_df = pd.merge(features_df, label_df, left_index=True, right_index=True, how='inner')
#features_label_df = pd.merge(features_df, label_df[['label']], left_index=True, right_index=True, how='inner')

# 保存合并后的 DataFrame 到一个新的 CSV 文件
features_label_df.to_csv('stock_1d_features_label.csv')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# 假设 features_label_df 是包含您数据的 DataFrame
features_label_df.to_csv('/content/drive/My Drive/stock_1d_features_label.csv', index=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
print(features_label_df.shape)

(901, 520)


In [8]:
!pip install numpy==1.24.3
!pip install pandas_ta

In [9]:
import numpy as np  # 导入numpy库，用于数值计算
import pandas as pd  # 导入pandas库，用于数据处理
import plotly.graph_objects as go  # 导入plotly.graph_objects库，用于创建图形对象
from plotly.subplots import make_subplots  # 导入plotly.subplots库，用于创建子图
from dataclasses import dataclass  # 导入dataclasses库，用于创建数据类
from typing import List, Tuple, Optional, Dict  # 导入typing库，用于类型注解
import pandas_ta as ta  # 导入pandas_ta库，用于技术分析指标计算

@dataclass  # 使用dataclass装饰器创建数据类
class FractalPoint:
    """分型点数据类"""
    type: int  # 1: 顶分型, -1: 底分型, 0: 起始/结束点
    time: str  # 分型点时间
    price: float  # 分型点价格
    offset: int  # 分型点偏移量

@dataclass  # 使用dataclass装饰器创建数据类
class PivotRange:
    """中枢范围数据类"""
    x_range: Tuple[float, float]  # 时间范围
    price_range: Tuple[float, float]  # 价格范围

class TechnicalAnalysis:
    """技术分析类"""

    def __init__(self, data: pd.DataFrame):
        """初始化技术分析类"""
        self.validate_data(data)  # 验证输入数据
        self.data = data.copy()  # 复制输入数据，避免修改原始数据
        # 保持原始列名不变
        self.column_map = {  # 定义列名映射字典
            'high': 'High',
            'low': 'Low',
            'open': 'Open',
            'close': 'Close',
            'volume': 'Volume'
        }

    @staticmethod
    def validate_data(data: pd.DataFrame) -> None:
        """验证输入数据的完整性"""
        required_columns = ['High', 'Low', 'Open', 'Close', 'Volume']  # 定义必需的列名
        if not all(col in data.columns for col in required_columns):  # 检查所有必需的列是否存在
            raise ValueError(f"Data must contain columns: {required_columns}")  # 如果缺少列，则抛出异常
        if len(data) < 3:  # 检查数据长度是否足够
            raise ValueError("Need at least 3 k-lines for analysis")  # 如果数据长度不足，则抛出异常

    def calculate_technical_indicators(self) -> Dict[str, pd.Series]:
        """计算技术指标"""
        # 创建一个临时DataFrame，列名为小写
        temp_data = self.data.copy()  # 复制数据，避免修改原始数据
        temp_data.columns = [col.lower() for col in temp_data.columns]  # 将列名转换为小写

        indicators = {}  # 初始化指标字典

        # MACD
        macd = temp_data.ta.macd(close='close', fast=12, slow=26, signal=9)  # 计算MACD指标
        indicators['macd'] = macd['MACD_12_26_9']  # 提取MACD线
        indicators['macdsignal'] = macd['MACDs_12_26_9']  # 提取MACD信号线
        indicators['macdhist'] = macd['MACDh_12_26_9']  # 提取MACD柱状图

        # KDJ
        stoch = temp_data.ta.stoch(high='high', low='low', close='close', k=9, d=3, smooth_k=3)  # 计算KDJ指标
        indicators['kdj_k'] = stoch['STOCHk_9_3_3']  # 提取K值
        indicators['kdj_d'] = stoch['STOCHd_9_3_3']  # 提取D值
        indicators['kdj_j'] = 3 * indicators['kdj_k'] - 2 * indicators['kdj_d']  # 计算J值

        # MA
        for period in [5, 10, 20, 30, 60]:  # 遍历移动平均线周期
            indicators[f'ma{period}'] = temp_data.ta.sma(close='close', length=period)  # 计算移动平均线

        return indicators  # 返回指标字典

    def adjust_by_containment(self, k_data: pd.DataFrame = None) -> pd.DataFrame:
        """处理K线包含关系"""
        if k_data is None:  # 如果没有提供K线数据，则使用原始数据
            k_data = self.data.copy()  # 复制原始数据

        result = []  # 初始化结果列表
        curr_k = k_data.iloc[0].copy()  # 初始化当前K线
        trend = 0  # 初始化趋势

        for i in range(1, len(k_data)):  # 遍历K线数据
            next_k = k_data.iloc[i]  # 获取下一根K线
            contained = (  # 判断是否存在包含关系
                (curr_k['High'] >= next_k['High'] and curr_k['Low'] <= next_k['Low']) or
                (curr_k['High'] <= next_k['High'] and curr_k['Low'] >= next_k['Low'])
            )

            if contained:  # 如果存在包含关系
                if trend >= 0:  # 如果趋势向上或初始状态
                    curr_k['High'] = max(curr_k['High'], next_k['High'])  # 更新当前K线最高价
                    curr_k['Low'] = max(curr_k['Low'], next_k['Low'])  # 更新当前K线最低价
                else:  # 如果趋势向下
                    curr_k['High'] = min(curr_k['High'], next_k['High'])  # 更新当前K线最高价
                    curr_k['Low'] = min(curr_k['Low'], next_k['Low'])  # 更新当前K线最低价
            else:  # 如果不存在包含关系
                result.append({  # 添加当前K线到结果列表
                    'Open': curr_k['Open'],
                    'High': curr_k['High'],
                    'Low': curr_k['Low'],
                    'Close': curr_k['Close'],
                    'Volume': curr_k['Volume'],
                    'Date': k_data.index[i-1]
                })

                if next_k['High'] > curr_k['High'] and next_k['Low'] > curr_k['Low']:  # 如果下一根K线高于当前K线
                    trend = 1  # 设置趋势为向上
                elif next_k['High'] < curr_k['High'] and next_k['Low'] < curr_k['Low']:  # 如果下一根K线低于当前K线
                    trend = -1  # 设置趋势为向下

                curr_k = next_k.copy()  # 更新当前K线

        result.append({  # 添加最后一根K线到结果列表
            'Open': curr_k['Open'],
            'High': curr_k['High'],
            'Low': curr_k['Low'],
            'Close': curr_k['Close'],
            'Volume': curr_k['Volume'],
            'Date': k_data.index[-1]
        })

        df = pd.DataFrame(result)  # 将结果列表转换为DataFrame
        df.set_index('Date', inplace=True)  # 设置日期为索引
        return df  # 返回处理后的K线数据

    def get_fx(self, k_data: pd.DataFrame = None) -> Tuple[List[int], List[str], pd.DataFrame, List[float], List[int]]:
        """识别分型"""
        if k_data is None:  # 如果没有提供K线数据，则使用处理后的K线数据
            k_data = self.adjust_by_containment()  # 使用处理后的K线数据

        fractals = []  # 初始化分型列表

        # 添加起始点
        fractals.append(FractalPoint(  # 添加起始分型点
            type=0,
            time=k_data.index[0].strftime("%Y-%m-%d %H:%M:%S"),
            price=(k_data['Low'].iloc[0] + k_data['High'].iloc[0]) / 2,
            offset=0
        ))

        # 识别分型
        for i in range(1, len(k_data)-1):  # 遍历K线数据
            # 正确获取三根K线
            k1 = k_data.iloc[i-1]
            k2 = k_data.iloc[i]
            k3 = k_data.iloc[i+1]

            # 顶分型
            if (k1['High'] <= k2['High'] and k2['High'] > k3['High']):  # 如果当前K线最高价高于前后K线最高价
                fractals.append(FractalPoint(  # 添加顶分型点
                    type=1,
                    time=k_data.index[i].strftime("%Y-%m-%d %H:%M:%S"),
                    price=k2['High'],
                    offset=i
                ))

            # 底分型
            elif (k1['Low'] >= k2['Low'] and k2['Low'] < k3['Low']):  # 如果当前K线最低价低于前后K线最低价
                fractals.append(FractalPoint(  # 添加底分型点
                    type=-1,
                    time=k_data.index[i].strftime("%Y-%m-%d %H:%M:%S"),
                    price=k2['Low'],
                    offset=i
                ))

        # 添加结束点
        fractals.append(FractalPoint(  # 添加结束分型点
            type=0,
            time=k_data.index[-1].strftime("%Y-%m-%d %H:%M:%S"),
            price=(k_data['Low'].iloc[-1] + k_data['High'].iloc[-1]) / 2,
            offset=len(k_data)-1
        ))

        # 构建返回数据
        fx_type = [f.type for f in fractals]  # 获取分型类型列表
        fx_time = [f.time for f in fractals]  # 获取分型时间列表
        fx_data = k_data.loc[[k_data.index[f.offset] for f in fractals]]  # 获取分型数据DataFrame
        fx_plot = [f.price for f in fractals]  # 获取分型价格列表
        fx_offset = [f.offset for f in fractals]  # 获取分型偏移列表

        return fx_type, fx_time, fx_data, fx_plot, fx_offset  # 返回分型类型、时间、数据、价格、偏移列表

class PlotlyChartPlotter:
    def __init__(self, k_data: pd.DataFrame, indicators: Dict[str, pd.Series] = None):
        self.k_data = k_data.copy()  # 复制K线数据
        self.indicators = indicators or {}  # 复制指标数据，如果没有提供则初始化为空字典
        #print(k_data)
        #print(indicators)

    def create_candlestick_chart(self) -> go.Figure:
        """创建K线图"""
        # 创建子图
        fig = make_subplots(  # 创建子图
            rows=4,  # 设置行数
            cols=1,  # 设置列数
            shared_xaxes=True,  # 设置共享x轴
            vertical_spacing=0.02,  # 设置垂直间距
            row_heights=[0.5, 0.2, 0.15, 0.15],  # 设置行高比例
            subplot_titles=('K线图', '成交量', 'MACD', 'KDJ')  # 设置子图标题
        )

        # 添加K线图
        fig.add_trace(  # 添加K线图
            go.Candlestick(  # 创建K线图对象
                x=self.k_data.index,  # 设置x轴数据
                open=self.k_data['Open'],  # 设置开盘价
                high=self.k_data['High'],  # 设置最高价
                low=self.k_data['Low'],  # 设置最低价
                close=self.k_data['Close'],  # 设置收盘价
                name='K线'  # 设置图例名称
            ),
            row=1, col=1  # 设置子图位置
        )

        # 添加MA线
        for period in [5, 10, 20, 30, 60]:  # 遍历移动平均线周期
            if f'ma{period}' in self.indicators:  # 如果存在该周期的移动平均线
                fig.add_trace(  # 添加移动平均线
                    go.Scatter(  # 创建折线图对象
                        x=self.k_data.index,  # 设置x轴数据
                        y=self.indicators[f'ma{period}'],  # 设置y轴数据
                        name=f'MA{period}',  # 设置图例名称
                        line=dict(width=1)  # 设置线宽
                    ),
                    row=1, col=1  # 设置子图位置
                )

        # 添加成交量图
        colors = ['red' if close >= open_ else 'green'  # 设置成交量柱状图颜色
                for close, open_ in zip(self.k_data['Close'], self.k_data['Open'])]

        fig.add_trace(  # 添加成交量图
            go.Bar(  # 创建柱状图对象
                x=self.k_data.index,  # 设置x轴数据
                y=self.k_data['Volume'],  # 设置y轴数据
                name='成交量',  # 设置图例名称
                marker_color=colors,  # 设置柱状图颜色
                opacity=0.5  # 设置透明度
            ),
            row=2, col=1  # 设置子图位置
        )

        # 添加MACD
        if all(k in self.indicators for k in ['macd', 'macdsignal', 'macdhist']):  # 如果存在MACD相关指标
            fig.add_trace(  # 添加MACD线
                go.Scatter(  # 创建折线图对象
                    x=self.k_data.index,  # 设置x轴数据
                    y=self.indicators['macd'],  # 设置y轴数据
                    name='MACD',  # 设置图例名称
                    line=dict(color='blue', width=1)  # 设置线条颜色和宽度
                ),
                row=3, col=1  # 设置子图位置
            )

            fig.add_trace(  # 添加MACD信号线
                go.Scatter(  # 创建折线图对象
                    x=self.k_data.index,  # 设置x轴数据
                    y=self.indicators['macdsignal'],  # 设置y轴数据
                    name='MACD Signal',  # 设置图例名称
                    line=dict(color='orange', width=1)  # 设置线条颜色和宽度
                ),
                row=3, col=1  # 设置子图位置
            )

            colors = ['red' if v >= 0 else 'green' for v in self.indicators['macdhist']]  # 设置MACD柱状图颜色
            fig.add_trace(  # 添加MACD柱状图
                go.Bar(  # 创建柱状图对象
                    x=self.k_data.index,  # 设置x轴数据
                    y=self.indicators['macdhist'],  # 设置y轴数据
                    name='MACD Hist',  # 设置图例名称
                    marker_color=colors  # 设置柱状图颜色
                ),
                row=3, col=1  # 设置子图位置
            )

        # 添加KDJ
        if all(k in self.indicators for k in ['kdj_k', 'kdj_d', 'kdj_j']):  # 如果存在KDJ相关指标
            fig.add_trace(  # 添加K值线
                go.Scatter(  # 创建折线图对象
                    x=self.k_data.index,  # 设置x轴数据
                    y=self.indicators['kdj_k'],  # 设置y轴数据
                    name='K值',  # 设置图例名称
                    line=dict(color='blue', width=1)  # 设置线条颜色和宽度
                ),
                row=4, col=1  # 设置子图位置
            )

            fig.add_trace(  # 添加D值线
                go.Scatter(  # 创建折线图对象
                    x=self.k_data.index,  # 设置x轴数据
                    y=self.indicators['kdj_d'],  # 设置y轴数据
                    name='D值',  # 设置图例名称
                    line=dict(color='orange', width=1)  # 设置线条颜色和宽度
                ),
                row=4, col=1  # 设置子图位置
            )

            fig.add_trace(  #添加J值线
                go.Scatter(  # 创建折线图对象
                    x=self.k_data.index,  # 设置x轴数据
                    y=self.indicators['kdj_j'],  # 设置y轴数据
                    name='J值',  # 设置图例名称
                    line=dict(color='purple', width=1)  # 设置线条颜色和宽度
                ),
                row=4, col=1  # 设置子图位置
            )

        # 更新布局
        fig.update_layout(  # 更新布局
            title='股票技术分析图',  # 设置标题
            xaxis_rangeslider_visible=False,  # 隐藏x轴滑动条
            height=1000,  # 设置高度
            showlegend=True,  # 显示图例
            legend=dict(  # 设置图例位置
                orientation="h",  # 设置图例方向
                yanchor="bottom",  # 设置图例y轴锚点
                y=1.02,  # 设置图例y轴位置
                xanchor="right",  # 设置图例x轴锚点
                x=1  # 设置图例x轴位置
            )
        )

        # 更新Y轴标题
        fig.update_yaxes(title_text="价格", row=1, col=1)  # 设置K线图y轴标题
        fig.update_yaxes(title_text="成交量", row=2, col=1)  # 设置成交量图y轴标题
        fig.update_yaxes(title_text="MACD", row=3, col=1)  # 设置MACD图y轴标题
        fig.update_yaxes(title_text="KDJ", row=4, col=1)  # 设置KDJ图y轴标题

        return fig  # 返回图形对象

    def add_fractals(self, fig: go.Figure,
                    fx_plot: List[float],
                    fx_time: List[str],
                    fx_type: List[int]) -> None:
        """添加分型标记"""
        fx_datetime = [pd.to_datetime(t) for t in fx_time]  # 将分型时间转换为datetime对象

        # 添加顶分型标记
        tops = [(t, p) for t, p, type_ in zip(fx_datetime, fx_plot, fx_type) if type_ == 1]  # 获取顶分型点
        if tops:  # 如果存在顶分型点
            top_times, top_prices = zip(*tops)  # 解压顶分型点
            fig.add_trace(  # 添加顶分型标记
                go.Scatter(  # 创建散点图对象
                    x=top_times,  # 设置x轴数据
                    y=top_prices,  # 设置y轴数据
                    mode='markers',  # 设置标记模式
                    marker=dict(  # 设置标记样式
                        symbol='triangle-down',  # 设置标记符号
                        size=10,  # 设置标记大小
                        color='red'  # 设置标记颜色
                    ),
                    name='顶分型'  # 设置图例名称
                ),
                row=1, col=1  # 设置子图位置
            )

        # 添加底分型标记
        bottoms = [(t, p) for t, p, type_ in zip(fx_datetime, fx_plot, fx_type) if type_ == -1]  # 获取底分型点
        if bottoms:  # 如果存在底分型点
            bottom_times, bottom_prices = zip(*bottoms)  # 解压底分型点
            fig.add_trace(  # 添加底分型标记
                go.Scatter(  # 创建散点图对象
                    x=bottom_times,  # 设置x轴数据
                    y=bottom_prices,  # 设置y轴数据
                    mode='markers',  # 设置标记模式
                    marker=dict(  # 设置标记样式
                        symbol='triangle-up',  # 设置标记符号
                        size=10,  # 设置标记大小
                        color='green'  # 设置标记颜色
                    ),
                    name='底分型'  # 设置图例名称
                ),
                row=1, col=1  # 设置子图位置
            )

        # 添加分型连线
        fig.add_trace(  # 添加分型连线
            go.Scatter(  # 创建折线图对象
                x=fx_datetime,  # 设置x轴数据
                y=fx_plot,  # 设置y轴数据
                mode='lines',  # 设置线条模式
                line=dict(  # 设置线条样式
                    color='blue',  # 设置线条颜色
                    width=1  # 设置线条宽度
                ),
                name='分型连线'  # 设置图例名称
            ),
            row=1, col=1  # 设置子图位置
        )

    def add_pivot(self, fig: go.Figure, pivot: PivotRange) -> None:
        """添加中枢区间"""
        if pivot.x_range[0] == 0 and pivot.x_range[1] == 0:  # 如果中枢无效
            return  # 直接返回

        x_dates = self.k_data.index  # 获取K线日期
        x_start = x_dates[int(pivot.x_range[0])]  # 获取中枢开始日期
        x_end = x_dates[int(pivot.x_range[1])]  # 获取中枢结束日期

        fig.add_trace(  # 添加中枢区间
            go.Scatter(  # 创建填充区域对象
                x=[x_start, x_start, x_end, x_end, x_start],  # 设置x轴数据
                y=[pivot.price_range[0], pivot.price_range[1],  # 设置y轴数据
                   pivot.price_range[1], pivot.price_range[0],
                   pivot.price_range[0]],
                fill="toself",  # 设置填充模式
                fillcolor="rgba(0,176,246,0.2)",  # 设置填充颜色
                line=dict(color="rgba(0,176,246,0.2)"),  # 设置边框颜色
                name="中枢区间"  # 设置图例名称
            ),
            row=1, col=1  # 设置子图位置
        )

def calculate_pivot(fx_plot: List[float], fx_offset: List[int], fx_observe: int) -> PivotRange:
    """计算中枢"""
    if fx_observe < 1:  # 如果观测点无效
        return PivotRange((0, 0), (0, 0))  # 返回无效中枢

    right_bound = (fx_offset[fx_observe] + fx_offset[fx_observe - 1]) / 2  # 计算右边界
    left_bound = 0  # 初始化左边界
    min_high = max(fx_plot[fx_observe], fx_plot[fx_observe - 1])  # 初始化最小高点
    max_low = min(fx_plot[fx_observe], fx_plot[fx_observe - 1])  # 初始化最大低点

    count = 0  # 初始化覆盖次数
    i = fx_observe - 1  # 初始化遍历索引

    while i >= 1:  # 遍历分型点
        curr_high = max(fx_plot[i], fx_plot[i - 1])  # 获取当前高点
        curr_low = min(fx_plot[i], fx_plot[i - 1])  # 获取当前低点

        if curr_high < max_low or curr_low > min_high:  # 如果当前高点低于最大低点或当前低点高于最小高点
            left_bound = (fx_offset[i] + fx_offset[i + 1]) / 2  # 计算左边界
            break  # 跳出循环

        min_high = min(min_high, curr_high)  # 更新最小高点
        max_low = max(max_low, curr_low)  # 更新最大低点
        count += 1  # 覆盖次数加1
        i -= 1  # 索引减1

    if count < 3:  # 如果覆盖次数小于3
        return PivotRange((0, 0), (0, 0))  # 返回无效中枢

    return PivotRange((left_bound, right_bound), (max_low, min_high))  # 返回中枢范围

def main():
    try:
        # 读取数据
        data = pd.read_csv('stock_1d.csv',
                          index_col='Date',
                          parse_dates=True)

        print("数据加载成功，样本数:", len(data))
        print("\n数据前5行:")
        print(data.head())

        # 创建技术分析对象
        ta = TechnicalAnalysis(data)

        # 计算技术指标
        indicators = ta.calculate_technical_indicators()

        # 处理包含关系
        print("\n处理包含关系...")
        adjusted_k_data = ta.adjust_by_containment(data)

        # 识别分型
        print("\n识别分型...")
        fx_type, fx_time, fx_data, fx_plot, fx_offset = ta.get_fx(adjusted_k_data)

        # 创建Plotly图表
        print("\n创建图表...")
        #plotter = PlotlyChartPlotter(adjusted_k_data, indicators)
        plotter = PlotlyChartPlotter(data, indicators)

        # 创建基础K线图
        fig = plotter.create_candlestick_chart()

        # 添加分型
        plotter.add_fractals(fig, fx_plot, fx_time, fx_type)

        # 计算并添加中枢
        pivot = calculate_pivot(fx_plot, fx_offset, len(fx_offset) - 2)
        plotter.add_pivot(fig, pivot)

        # 显示图表
        fig.show()

    except FileNotFoundError:
        print("错误: 未找到文件 'stock_1d.csv'，请确保文件存在于正确的路径")
    except Exception as e:
        print(f"分析过程中出现错误: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

数据加载成功，样本数: 961

数据前5行:
                Close       High        Low       Open     Volume
Date                                                             
2020-01-02  72.716064  72.776591  71.466805  71.721011  135480400
2020-01-03  72.009125  72.771752  71.783969  71.941336  146322800
2020-01-06  72.582901  72.621639  70.876068  71.127858  118387200
2020-01-07  72.241547  72.849224  72.021231  72.592594  108872000
2020-01-08  73.403641  73.706271  71.943751  71.943751  132079200

处理包含关系...

识别分型...

创建图表...
